<a href="https://colab.research.google.com/github/Theo-Daniel/MSc-Thesis-DataScience/blob/main/Copy_of_Thesis_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DATASET

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd

cancer_df = pd.read_csv('/content/drive/My Drive/Thesis2/Cancer.txt', sep='\t', index_col=0)
normal_df = pd.read_csv('/content/drive/My Drive/Thesis2/Normal.txt', sep='\t', index_col=0)

# PREPROCESSING

## TRANSPOSE (samples = rows, genes = columns), TARGET LABELS

In [ ]:
cancer_df = cancer_df.T  # transpose: samples = rows, genes = columns
cancer_df['label'] = 1   # mark as cancer

normal_df = normal_df.T
normal_df['label'] = 0   # mark as normal

# Combine the two datasets
combined_df = pd.concat([cancer_df, normal_df], axis=0)

# Split into features and labels
X = combined_df.drop('label', axis=1)
y = combined_df['label']

print(f"Dataset shape: {X.shape}, Labels: {y.value_counts().to_dict()}")

In [ ]:
# Create backup copies of features and labels
X_orig = X.copy()
y_orig = y.copy()

## DATASET ANALYSIS, LOG TRANSFORM, NORMALISATION

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 1) Check for missing values
missing_counts = X.isnull().sum().sum()
print(f"Total missing values: {missing_counts:,}")

# (Optional) Impute / drop if needed
# X = X.fillna(0)
# X = X.dropna()

# 2) Basic statistics (raw scale)
print("Before transformation:")
print(X.describe().T[['mean', 'std', 'min', 'max']].head())

# 3) Clean histograms for the first 5 genes (no overlap)
first_cols = list(X.columns[:5])
fig_axes = X[first_cols].hist(
    bins=50,
    layout=(2, 3),       # 2 rows x 3 cols -> room for 5 plots
    figsize=(12, 6),
    sharex=False,
    sharey=False,
    grid=False
)
axes = np.asarray(fig_axes)

# Remove any unused subplot (since we created a 2x3 grid for 5 plots)
for ax in axes.ravel()[len(first_cols):]:
    ax.remove()

# Add a neat overall title and tighten layout
fig = axes.ravel()[0].get_figure()
fig.suptitle("Raw Expression Distributions (First 5 Genes)", y=1.02)
fig.tight_layout()
plt.show()

# 4) Log transform if clearly not already in log scale
if X.values.max() > 100:
    X = np.log2(X + 1)
    print("Applied log2(x + 1) transformation.")

# 5) Standardize features (mean=0, std=1) using TRAIN fit only in your pipeline
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 6) Back to DataFrame (optional)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

print("After standardization:")
print(X_scaled_df.describe().T[['mean', 'std']].head())

# FEATURE SELECTION

## 1. LASSO

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Define stratified CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Fit Lasso (L1) Logistic Regression with CV
lasso = LogisticRegressionCV(
    Cs=[0.01, 0.1, 1],
    penalty='l1',
    solver='saga',
    max_iter=5000,
    cv=cv,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)
lasso.fit(X_scaled, y)

# Get absolute coefficient values
coefs = np.abs(lasso.coef_[0])
gene_names = X.columns

# Create a DataFrame of genes and their Lasso coefficients
lasso_df = pd.DataFrame({'gene': gene_names, 'coef': coefs})
lasso_df = lasso_df[lasso_df['coef'] > 0]  # Keep only non-zero
lasso_df = lasso_df.sort_values(by='coef', ascending=False).head(50)

# Get the top 50 genes
top_50_genes_lasso = lasso_df['gene'].tolist()

print("Top 5 genes by Lasso:")
print(lasso_df.head())

# Subset original scaled X to only top 50 features
X_lasso_top50 = X_scaled_df[top_50_genes_lasso]

## 2. RFE

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Initialize Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Set up Stratified K-Fold (5 splits)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize array to hold support masks across folds
support_matrix = np.zeros(X_scaled_df.shape[1])

# Loop through each fold and apply RFE
for train_idx, val_idx in skf.split(X_scaled_df, y):
    X_train, y_train = X_scaled_df.iloc[train_idx], y.iloc[train_idx]

    # Apply RFE on training set only
    rfe = RFE(estimator=rf, n_features_to_select=50, step=0.05)
    rfe.fit(X_train, y_train)

    # Add boolean mask (True = selected) to matrix
    support_matrix += rfe.support_

# Rank genes by frequency of selection across folds
feature_counts = pd.Series(support_matrix, index=X_scaled_df.columns)
top_50_genes_rfe = feature_counts.sort_values(ascending=False).head(50).index.tolist()

# Print top 5 genes
print("Top 5 genes by RFE with stratified sampling:")
print(top_50_genes_rfe[:5])

# Create final reduced feature matrix
X_rfe_top50 = X_scaled_df[top_50_genes_rfe]

## SAVE AS CSV

In [ ]:
import pandas as pd

# ------------------- STEP 1: Define reusable function -------------------
def create_selected_gene_df_with_values(gene_list, value_dict, scaled_df):
    """
    Given a list of selected genes, a dictionary of gene:value (importance),
    and the original scaled DataFrame, return:
    - selected_df: subset with only selected genes
    - value_series: Series of importance values indexed by gene
    """
    selected_df = scaled_df[gene_list].copy()
    value_series = pd.Series({gene: value_dict[gene] for gene in gene_list})
    return selected_df, value_series

# ------------------- STEP 2: Lasso output -------------------
# Assuming: top_50_genes_lasso, lasso_df, X_scaled_df already exist

# Create dictionary: gene -> coefficient
lasso_coef_dict = dict(zip(lasso_df['gene'], lasso_df['coef']))

# Create reduced DataFrame and coefficient series
X_lasso_top50_df, lasso_coef_series = create_selected_gene_df_with_values(
    top_50_genes_lasso, lasso_coef_dict, X_scaled_df
)

# Save to CSV:
X_lasso_top50_df.to_csv("Lasso_Selected_Genes.csv")
lasso_coef_series.to_csv("Lasso_Coefficients.csv")

# ------------------- STEP 3: RFE output -------------------
# Assuming: top_50_genes_rfe, feature_counts, X_scaled_df already exist

# Create dictionary: gene -> selection count
rfe_importance_dict = feature_counts.to_dict()

# Create reduced DataFrame and selection frequency series
X_rfe_top50_df, rfe_importance_series = create_selected_gene_df_with_values(
    top_50_genes_rfe, rfe_importance_dict, X_scaled_df
)

# Save to CSV:
X_rfe_top50_df.to_csv("RFE_Selected_Genes.csv")
rfe_importance_series.to_csv("RFE_SelectionFrequency.csv")

## LASSO, RFE SELECTED FEATURES

In [ ]:
print("Lasso - Top 10 Genes (Expression Values):")
print(X_lasso_top50_df.head(10))

print("\nLasso - Top 10 Gene Coefficients:")
print(lasso_coef_series.head(10))

In [ ]:
print("RFE - Top 10 Genes (Expression Values):")
print(X_rfe_top50_df.head(10))

print("\nRFE - Top 10 Gene Selection Frequencies:")
print(rfe_importance_series.head(10))

# BINARY CLASSIFICATION (Tumour VS Normal)

## 1. LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import pandas as pd

# Split both Lasso & RFE feature sets using stratified sampling
X_train_lasso, X_test_lasso, y_train, y_test = train_test_split(
    X_lasso_top50, y, test_size=0.2, stratify=y, random_state=42
)

X_train_rfe, X_test_rfe, _, _ = train_test_split(
    X_rfe_top50, y, test_size=0.2, stratify=y, random_state=42
)

# Evaluation function (includes prediction breakdown)
def evaluate_model(model, X_test, y_test, name="Model"):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]  # Class 1 (cancer) probability

    print(f"\nEvaluation Report for {name}")
    print("Accuracy :", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall   :", recall_score(y_test, y_pred))
    print("F1 Score :", f1_score(y_test, y_pred))
    print("ROC AUC  :", roc_auc_score(y_test, y_proba))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("\n🔬 Prediction breakdown (first 20 samples):")
    breakdown = pd.DataFrame({
        'Actual Label': y_test.values,
        'Predicted Label': y_pred,
        'Cancer Prob (class=1)': y_proba
    })
    print(breakdown.head(20))

# Train & evaluate Logistic Regression on Lasso features
lr_lasso = LogisticRegression(max_iter=5000, random_state=42)
lr_lasso.fit(X_train_lasso, y_train)
evaluate_model(lr_lasso, X_test_lasso, y_test, name="Logistic Regression (Lasso Features)")

# Train & evaluate Logistic Regression on RFE features
lr_rfe = LogisticRegression(max_iter=5000, random_state=42)
lr_rfe.fit(X_train_rfe, y_train)
evaluate_model(lr_rfe, X_test_rfe, y_test, name="Logistic Regression (RFE Features)")

## 2. SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

# Helper function to evaluate models
def evaluate_model(model, X_test, y_test, name="Model"):
    y_pred = model.predict(X_test)
    y_proba = model.decision_function(X_test)  # SVM: use decision_function for probabilities

    print(f"\n Evaluation Report for {name}")
    print("Accuracy :", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall   :", recall_score(y_test, y_pred))
    print("F1 Score :", f1_score(y_test, y_pred))
    print("ROC AUC  :", roc_auc_score(y_test, y_proba))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Print predicted labels alongside actual labels
    print("\n Prediction breakdown (first 20 samples):")
    results_df = pd.DataFrame({
        'Actual Label': y_test.values,
        'Predicted Label': y_pred,
        'Decision Score (Cancer=1)': y_proba
    })
    print(results_df.head(20))

# SVM with Lasso Features
svm_lasso = SVC(kernel='linear', probability=True, random_state=42)
svm_lasso.fit(X_train_lasso, y_train)
evaluate_model(svm_lasso, X_test_lasso, y_test, name="SVM (Lasso Features)")

# SVM with RFE Features
svm_rfe = SVC(kernel='linear', probability=True, random_state=42)
svm_rfe.fit(X_train_rfe, y_train)
evaluate_model(svm_rfe, X_test_rfe, y_test, name="SVM (RFE Features)")

## 3. RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train-test split using saved variables
X_train_lasso, X_test_lasso, y_train, y_test = train_test_split(
    X_lasso_top50_df, y, test_size=0.2, stratify=y, random_state=42
)

X_train_rfe, X_test_rfe, _, _ = train_test_split(
    X_rfe_top50_df, y, test_size=0.2, stratify=y, random_state=42
)


# Evaluation function
def evaluate_model(model, X_test, y_test, name="Model"):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    print(f"\nEvaluation Report for {name}")
    print("Accuracy :", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall   :", recall_score(y_test, y_pred))
    print("F1 Score :", f1_score(y_test, y_pred))
    print("ROC AUC  :", roc_auc_score(y_test, y_proba))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("\nPrediction breakdown (first 20 samples):")
    prediction_df = pd.DataFrame({
        'Actual Label': y_test.values,
        'Predicted Label': y_pred,
        'Cancer Probability (class=1)': y_proba
    })
    print(prediction_df.head(20))

# Train & evaluate Random Forest with Lasso features
rf_lasso = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_lasso.fit(X_train_lasso, y_train)
evaluate_model(rf_lasso, X_test_lasso, y_test, name="Random Forest (Lasso Features)")

# Train & evaluate Random Forest with RFE features
rf_rfe = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_rfe.fit(X_train_rfe, y_train)
evaluate_model(rf_rfe, X_test_rfe, y_test, name="Random Forest (RFE Features)")

## 4. XGBOOST

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

# Evaluation function (same for all models)
def evaluate_model(model, X_test, y_test, name="Model"):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    print(f"\n Evaluation Report for {name}")
    print("Accuracy :", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall   :", recall_score(y_test, y_pred))
    print("F1 Score :", f1_score(y_test, y_pred))
    print("ROC AUC  :", roc_auc_score(y_test, y_proba))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Prediction breakdown
    print("\n Prediction breakdown (first 20 samples):")
    results_df = pd.DataFrame({
        'Actual Label': y_test.values,
        'Predicted Label': y_pred,
        'Cancer Prob (class=1)': y_proba
    })
    print(results_df.head(20))


# 1. XGBoost with Lasso-selected features
xgb_lasso = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3,
                          random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_lasso.fit(X_train_lasso, y_train)
evaluate_model(xgb_lasso, X_test_lasso, y_test, name="XGBoost (Lasso Features)")


# 2. XGBoost with RFE-selected features
xgb_rfe = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3,
                        random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_rfe.fit(X_train_rfe, y_train)
evaluate_model(xgb_rfe, X_test_rfe, y_test, name="XGBoost (RFE Features)")

## 5. MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import pandas as pd

# Evaluation function (same as before)
def evaluate_model(model, X_test, y_test, name="Model"):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    print(f"\n Evaluation Report for {name}")
    print("Accuracy :", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred))
    print("Recall   :", recall_score(y_test, y_pred))
    print("F1 Score :", f1_score(y_test, y_pred))
    print("ROC AUC  :", roc_auc_score(y_test, y_proba))
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    # Sample-wise prediction breakdown
    print("\n Prediction breakdown (first 20 samples):")
    results_df = pd.DataFrame({
        'Actual Label': y_test.values,
        'Predicted Label': y_pred,
        'Cancer Probability (class=1)': y_proba
    })
    print(results_df.head(20))


# Train MLP on Lasso-selected features
mlp_lasso = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam',
                          max_iter=500, random_state=42)
mlp_lasso.fit(X_train_lasso, y_train)
evaluate_model(mlp_lasso, X_test_lasso, y_test, name="MLP Classifier (Lasso Features)")


# Train MLP on RFE-selected features
mlp_rfe = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam',
                        max_iter=500, random_state=42)
mlp_rfe.fit(X_train_rfe, y_train)
evaluate_model(mlp_rfe, X_test_rfe, y_test, name="MLP Classifier (RFE Features)")

# EXPLAINABLE ML

# 1. SHAPLEY

## 1(A). SHAP - Logistic Regression x LASSO50

In [ ]:
# --- SHAP for Logistic Regression (LASSO) ---
import shap
import numpy as np
import matplotlib.pyplot as plt

# small background to estimate the expectation (keeps it fast & avoids leakage)
bg_lasso = shap.sample(X_train_lasso, 300, random_state=42)

# LinearExplainer is the correct choice for (penalized) linear models
expl_lr_lasso = shap.LinearExplainer(lr_lasso, bg_lasso)

# SHAP values on the held-out TEST
sv_lasso = expl_lr_lasso.shap_values(X_test_lasso)

# If SHAP returns a list (some versions do), take the positive class (index 1)
if isinstance(sv_lasso, list):
    sv_lasso = sv_lasso[1]

# Plots: global importance (bar) + beeswarm
shap.summary_plot(sv_lasso, X_test_lasso, plot_type="bar", show=True)
shap.summary_plot(sv_lasso, X_test_lasso, show=True)

## 1(B). SHAP - Logistic Regression X RFE50

In [ ]:
# --- SHAP for Logistic Regression (RFE) ---
bg_rfe = shap.sample(X_train_rfe, 300, random_state=42)

expl_lr_rfe = shap.LinearExplainer(lr_rfe, bg_rfe)
sv_rfe = expl_lr_rfe.shap_values(X_test_rfe)

if isinstance(sv_rfe, list):
    sv_rfe = sv_rfe[1]

shap.summary_plot(sv_rfe, X_test_rfe, plot_type="bar", show=True)
shap.summary_plot(sv_rfe, X_test_rfe, show=True)

## 2(A). SHAP - SVM X LASSO50

In [ ]:
# --- SHAP for SVM (binary) — LASSO50 ---
import shap, numpy as np, pandas as pd

np.random.seed(42)  # for reproducibility of kmeans sampling

cols_lasso = X_train_lasso.columns

# summarize background to speed up Kernel SHAP
bg_lasso = shap.kmeans(X_train_lasso[cols_lasso], 100)   # <-- no random_state here

# wrapper: restore feature names before calling the model
def f_svm_lasso(X):
    X_df = pd.DataFrame(X, columns=cols_lasso)
    return svm_lasso.predict_proba(X_df)[:, 1]

expl_svm_lasso = shap.KernelExplainer(f_svm_lasso, bg_lasso, link="logit")

# optional: subsample TEST for speed
X_test_lasso_shap = X_test_lasso[cols_lasso].sample(800, random_state=42)

sv_lasso = expl_svm_lasso.shap_values(X_test_lasso_shap, nsamples="auto")

shap.summary_plot(sv_lasso, X_test_lasso_shap, plot_type="bar", show=True)
shap.summary_plot(sv_lasso, X_test_lasso_shap, show=True)

## 2(B). SHAP - SVM X RFE50

In [ ]:
# --- SHAP for SVM (binary) — RFE50 ---
np.random.seed(42)

cols_rfe = X_train_rfe.columns
bg_rfe = shap.kmeans(X_train_rfe[cols_rfe], 100)         # <-- no random_state

def f_svm_rfe(X):
    X_df = pd.DataFrame(X, columns=cols_rfe)
    return svm_rfe.predict_proba(X_df)[:, 1]

expl_svm_rfe = shap.KernelExplainer(f_svm_rfe, bg_rfe, link="logit")

X_test_rfe_shap = X_test_rfe[cols_rfe].sample(800, random_state=42)

sv_rfe = expl_svm_rfe.shap_values(X_test_rfe_shap, nsamples="auto")

shap.summary_plot(sv_rfe, X_test_rfe_shap, plot_type="bar", show=True)
shap.summary_plot(sv_rfe, X_test_rfe_shap, show=True)

## 3(A). SHAP - RF X LASSO50

In [ ]:
import shap
import matplotlib.pyplot as plt

# Initialize explainer
explainer_rf_lasso = shap.TreeExplainer(rf_lasso)

# Compute SHAP values (should be a 2D numpy array of shape [n_samples, n_features])
shap_values_lasso = explainer_rf_lasso.shap_values(X_test_lasso)

# If it returns a list, extract class 1 values (positive class)
if isinstance(shap_values_lasso, list):
    shap_values_lasso = shap_values_lasso[1]

# Plot SHAP summary
shap.summary_plot(shap_values_lasso, X_test_lasso, plot_type="bar")
shap.summary_plot(shap_values_lasso, X_test_lasso)

## 3(B). SHAP - RF X RFE50

In [ ]:
# Initialize SHAP explainer for RFE-selected Random Forest model
explainer_rf_rfe = shap.TreeExplainer(rf_rfe)

# Compute SHAP values for RFE test set
shap_values_rfe = explainer_rf_rfe.shap_values(X_test_rfe)

# If binary classification, extract class 1 SHAP values
if isinstance(shap_values_rfe, list):
    shap_values_rfe = shap_values_rfe[1]

# Plot SHAP summary (bar and beeswarm)
shap.summary_plot(shap_values_rfe, X_test_rfe, plot_type="bar")
shap.summary_plot(shap_values_rfe, X_test_rfe)

## 4(A). SHAP - XGBoost X LASSO50

In [ ]:
# ========== XGBoost with Lasso Features ==========
# Initialize SHAP explainer
explainer_xgb_lasso = shap.TreeExplainer(xgb_lasso)

# Compute SHAP values
shap_values_lasso = explainer_xgb_lasso.shap_values(X_test_lasso)

# If binary classification, extract class 1 SHAP values
if isinstance(shap_values_lasso, list):
    shap_values_lasso = shap_values_lasso[1]

# Plot SHAP summary (bar and beeswarm)
shap.summary_plot(shap_values_lasso, X_test_lasso, plot_type="bar")
shap.summary_plot(shap_values_lasso, X_test_lasso)

## 4(B). SHAP - XGBoost X RFE50

In [ ]:
# Initialize SHAP explainer
explainer_xgb_rfe = shap.TreeExplainer(xgb_rfe)

# Compute SHAP values
shap_values_rfe = explainer_xgb_rfe.shap_values(X_test_rfe)

# If binary classification, extract class 1 SHAP values
if isinstance(shap_values_rfe, list):
    shap_values_rfe = shap_values_rfe[1]

# Plot SHAP summary (bar and beeswarm)
shap.summary_plot(shap_values_rfe, X_test_rfe, plot_type="bar")
shap.summary_plot(shap_values_rfe, X_test_rfe)

## 5(A). SHAP - MLP X LASSO50

In [ ]:
# --- SHAP for MLP (binary) — LASSO50 ---
import shap, numpy as np, pandas as pd

cols_lasso = X_train_lasso.columns

# compact background to speed up Kernel SHAP
bg_lasso = shap.kmeans(X_train_lasso[cols_lasso], 100)  # or: shap.sample(..., 300)

# wrapper that restores feature names and returns P(class=1)
def f_mlp_lasso(X):
    X_df = pd.DataFrame(X, columns=cols_lasso)
    return mlp_lasso.predict_proba(X_df)[:, 1]

# model-agnostic explainer; use logit link so additivity is on log-odds
expl_mlp_lasso = shap.KernelExplainer(f_mlp_lasso, bg_lasso, link="logit")

# optional: subsample TEST to keep things snappy
X_test_lasso_shap = X_test_lasso[cols_lasso].sample(800, random_state=42)

# SHAP values
sv_lasso = expl_mlp_lasso.shap_values(X_test_lasso_shap, nsamples="auto")

# plots
shap.summary_plot(sv_lasso, X_test_lasso_shap, plot_type="bar", show=True)
shap.summary_plot(sv_lasso, X_test_lasso_shap, show=True)

## 5(B). SHAP - MLP X RFE50

In [ ]:
# --- SHAP for MLP (binary) — RFE50 ---
cols_rfe = X_train_rfe.columns
bg_rfe = shap.kmeans(X_train_rfe[cols_rfe], 100)

def f_mlp_rfe(X):
    X_df = pd.DataFrame(X, columns=cols_rfe)
    return mlp_rfe.predict_proba(X_df)[:, 1]

expl_mlp_rfe = shap.KernelExplainer(f_mlp_rfe, bg_rfe, link="logit")

X_test_rfe_shap = X_test_rfe[cols_rfe].sample(800, random_state=42)
sv_rfe = expl_mlp_rfe.shap_values(X_test_rfe_shap, nsamples="auto")

shap.summary_plot(sv_rfe, X_test_rfe_shap, plot_type="bar", show=True)
shap.summary_plot(sv_rfe, X_test_rfe_shap, show=True)

# 2. PDP + ICE

In [ ]:
# ==== PDP + ICE for all models (Lasso & RFE feature sets) ====
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
import numpy as np
import matplotlib.pyplot as plt

# --- helpers --------------------------------------------------
def _unwrap(est):
    return est.named_steps[list(est.named_steps)[-1]] if hasattr(est, "named_steps") else est

def _resp_method(est):
    return "predict_proba" if hasattr(est, "predict_proba") else "decision_function"

def _rank_features(estimator, X_ref, y_ref=None, k=8, fallback_names=None, use_perm=False):
    """Rank features by model-native importance/coef; optionally fallback to permutation importance."""
    fin = _unwrap(estimator)
    names = list(X_ref.columns) if hasattr(X_ref, "columns") else [f"f{i}" for i in range(X_ref.shape[1])]

    if hasattr(fin, "feature_importances_"):
        w = np.asarray(fin.feature_importances_)
    elif hasattr(fin, "coef_"):
        w = np.mean(np.abs(np.asarray(fin.coef_)), axis=0)  # binary/multiclass
    elif use_perm and (y_ref is not None):
        # permutation importance on the provided reference set (usually X_test_* for speed)
        pi = permutation_importance(estimator, X_ref, y_ref, n_repeats=5, random_state=42, scoring="roc_auc")
        w = pi.importances_mean
    else:
        # just take the first k if nothing else available
        return names[:k]

    order = np.argsort(w)[::-1][:k]
    return [names[i] for i in order]

def plot_pdp_block(model, X_ref, label, k=8, target_idx=1, grid=50, use_perm=False, y_ref=None, n_cols=2):
    feats = _rank_features(model, X_ref, y_ref=y_ref, k=k, use_perm=use_perm)

    disp = PartialDependenceDisplay.from_estimator(
        model,
        X_ref,
        features=feats,
        kind="both",
        grid_resolution=grid,
        target=target_idx,
        response_method=_resp_method(_unwrap(model)),
        n_jobs=-1
    )
    # Adjust layout: n_cols controls how many plots per row
    n_rows = int(np.ceil(len(feats) / n_cols))
    fig = plt.gcf()
    fig.set_size_inches(5 * n_cols, 3 * n_rows)
    fig.suptitle(f"{label} — PDP + ICE (top {len(feats)} features)", y=1.02, fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    return feats

## 1. PDP - Logistic Regression

In [ ]:
topk = 10

# --- Logistic Regression ---
lr_lasso_feats = plot_pdp_block(lr_lasso, X_train_lasso, "Logistic Regression (Lasso)", k=topk, target_idx=1)
lr_rfe_feats   = plot_pdp_block(lr_rfe,   X_train_rfe,   "Logistic Regression (RFE)",   k=topk, target_idx=1)

## 2. PDP - SVM

In [ ]:
# --- SVM (you trained with probability=True) ---
svm_lasso_feats = plot_pdp_block(svm_lasso, X_train_lasso, "SVM (Lasso)", k=topk, target_idx=1)
svm_rfe_feats   = plot_pdp_block(svm_rfe,   X_train_rfe,   "SVM (RFE)",   k=topk, target_idx=1)

## 3. PDP - Random Forest

In [ ]:
# --- Random Forest ---
rf_lasso_feats = plot_pdp_block(rf_lasso, X_train_lasso, "Random Forest (Lasso)", k=topk, target_idx=1)
rf_rfe_feats   = plot_pdp_block(rf_rfe,   X_train_rfe,   "Random Forest (RFE)",   k=topk, target_idx=1)

## 4. PDP - XGBoost

In [ ]:
# --- XGBoost ---
xgb_lasso_feats = plot_pdp_block(xgb_lasso, X_train_lasso, "XGBoost (Lasso)", k=topk, target_idx=1)
xgb_rfe_feats   = plot_pdp_block(xgb_rfe,   X_train_rfe,   "XGBoost (RFE)",   k=topk, target_idx=1)

## 5. PDP - MLP

In [ ]:
mlp_lasso_feats = plot_pdp_block(
    mlp_lasso, X_test_lasso, "MLP (Lasso)",
    k=topk, target_idx=1, use_perm=True, y_ref=y_test, n_cols=3
)
mlp_rfe_feats = plot_pdp_block(
    mlp_rfe, X_test_rfe, "MLP (RFE)",
    k=topk, target_idx=1, use_perm=True, y_ref=y_test, n_cols=3
)

# PAM 50 subtype classification (5 Class)

## Meta-Data

In [ ]:
import pandas as pd

scanb_xlsx_path = "/content/drive/My Drive/Thesis2/subtype.xlsx"

# peek sheets so we know which one to read
xlsx_handle = pd.ExcelFile(scanb_xlsx_path)
print("Sheets:", xlsx_handle.sheet_names)

In [ ]:
# pick the correct sheet name from the print above
scanb_sheet_name = xlsx_handle.sheet_names[0]   # <-- or e.g., "Merged"

# read; dtype=None lets pandas infer types, keep_default_na=True to parse blanks as NaN
scanb_meta_raw = pd.read_excel(scanb_xlsx_path, sheet_name=scanb_sheet_name)

# strip whitespace from column names
scanb_meta_raw.columns = scanb_meta_raw.columns.astype(str).str.strip()

print("Meta shape:", scanb_meta_raw.shape)
scanb_meta_raw.head(3)

In [ ]:
for col in id_candidates:
    print(col, list(map(str, scanb_meta_raw[col].dropna().astype(str).head(10))))

In [ ]:
# Inspect the ORIGINAL indices (not expr_by_sample_df)
print("X_scaled_df shape:", X_scaled_df.shape)
print("First 10 X_scaled_df indices:", list(map(str, X_scaled_df.index[:10])))

# If you still have `combined_df`, print that too (it should match X_scaled_df.index)
if 'combined_df' in globals():
    print("First 10 combined_df indices:", list(map(str, combined_df.index[:10])))

In [ ]:
# extract the core S####### id from your *row index*
core_series = extract_core_sid(X_scaled_df.index)
print("preview core IDs:", core_series.head(5).tolist())

expr_core_df = X_scaled_df.copy()
expr_core_df["__KEY__"] = core_series.values   # << critical: use .values (no alignment)
expr_core_df = (
    expr_core_df
      .dropna(subset=["__KEY__"])
      .set_index("__KEY__")
      .groupby(level=0).median()               # collapse multiple libs per sample
)

print("expr_core_df shape:", expr_core_df.shape)
print("first 5 core IDs:", expr_core_df.index[:5].tolist())

In [ ]:
# --- extract core sample IDs from the *row index* and collapse replicates ---

# 1) Get the core ID for each row (library); this returns an Index in our pipeline
core_idx = extract_core_sid(X_scaled_df.index)          # pd.Index (or Series)

# Preview a few IDs (Index doesn't have .head(), so slice instead)
print("preview core IDs:", list(map(str, core_idx[:5])))

# 2) Attach the core ID as a column and collapse multiple libraries per sample by median
expr_core_df = X_scaled_df.copy()
expr_core_df["__KEY__"] = pd.Index(core_idx).to_numpy()  # ensure 1-D array, no alignment issues
expr_core_df = (
    expr_core_df
      .dropna(subset=["__KEY__"])
      .set_index("__KEY__")
      .groupby(level=0).median()                         # collapse replicates
)

print("expr_core_df shape:", expr_core_df.shape)
print("first 5 core IDs:", list(map(str, expr_core_df.index[:5])))

In [ ]:
id_candidates = [c for c in ["Sample","RNA","Case","Patient"] if c in scanb_meta_raw.columns]
overlaps = {}
for col in id_candidates:
    meta_core = extract_core_sid(scanb_meta_raw[col])
    overlaps[col] = len(set(meta_core.dropna()) & set(expr_core_df.index))
    print(col, "overlap:", overlaps[col])

best_col = max(overlaps, key=overlaps.get)
print("Using metadata column:", best_col, "with overlap:", overlaps[best_col])

In [ ]:
pam50_label_col = "NCN.PAM50"

In [ ]:
meta_core_df = (
    scanb_meta_raw
      .dropna(subset=[best_col, pam50_label_col])
      .assign(__KEY__=extract_core_sid(scanb_meta_raw[best_col]).values)  # use .values
      .dropna(subset=["__KEY__"])
      .drop_duplicates("__KEY__")
      .set_index("__KEY__")
)

common_ids = meta_core_df.index.intersection(expr_core_df.index)
print("Common after core-ID normalization:", len(common_ids))

meta_aligned_df   = meta_core_df.loc[common_ids].copy()
expr_by_sample_df = expr_core_df.loc[common_ids].copy()

meta_aligned_df[[pam50_label_col, "Training.set"]].head()

In [ ]:
# sanity: same samples and same order
assert set(expr_by_sample_df.index) == set(meta_aligned_df.index)
expr_by_sample_df = expr_by_sample_df.loc[meta_aligned_df.index]  # enforce same order

print("Expr shape:", expr_by_sample_df.shape)
print("Meta shape:", meta_aligned_df.shape)
print("Label counts:\n", meta_aligned_df[pam50_label_col].value_counts())
print("Train/Test:\n", meta_aligned_df["Training.set"].value_counts())

# CLASSIFICATION MODELS

## Define the model:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

def model_zoo(num_classes):
    return {
        "LogReg": Pipeline([("z", StandardScaler()),
                            ("clf", LogisticRegression(max_iter=5000,
                                                       multi_class="multinomial",
                                                       class_weight="balanced",
                                                       random_state=42))]),
        "SVM":    Pipeline([("z", StandardScaler()),
                            ("clf", SVC(kernel="rbf",
                                        probability=True,
                                        class_weight="balanced",
                                        random_state=42))]),
        "RF":     RandomForestClassifier(n_estimators=400,
                                         class_weight="balanced",
                                         n_jobs=-1, random_state=42),
        "XGB":    XGBClassifier(objective="multi:softprob",
                                num_class=len(class_names),
                                n_estimators=500, learning_rate=0.1,
                                max_depth=6, subsample=0.9, colsample_bytree=0.9,
                                reg_lambda=1.0, n_jobs=-1, random_state=42,
                                eval_metric="mlogloss"),
        "MLP":    Pipeline([("z", StandardScaler()),
                            ("clf", MLPClassifier(hidden_layer_sizes=(128,),
                                                  activation="relu",
                                                  solver="adam",
                                                  max_iter=800,
                                                  random_state=42))]),
    }

## PAM50 feature matrices + targets + train/test

In [ ]:
# --- Build PAM50 feature matrices + targets + train/test ids ---

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pathlib import Path

# 0) assumes you already have:
#    expr_by_sample_df  (rows = sample IDs, cols = genes)
#    meta_aligned_df    (same index as expr_by_sample_df; has pam50 + Training.set)
#    pam50_label_col    (e.g., "NCN.PAM50")

# 1) Load/confirm 50-gene panels and subset expression
def load_gene_list(csv_path, df):
    try:
        cols = pd.read_csv(csv_path, nrows=0).columns.tolist()
    except Exception as e:
        print(f"Warning: could not read {csv_path}: {e}")
        cols = []
    # keep only genes present in the data
    return [c for c in cols if c in df.columns]

lasso_csv_path = "/content/drive/My Drive/Thesis2/Lasso_Selected_Genes.csv"
rfe_csv_path   = "/content/drive/My Drive/Thesis2/RFE_Selected_Genes.csv"

# allow fallback if these were already defined upstream
top_50_genes_lasso = globals().get("top_50_genes_lasso", []) or load_gene_list(lasso_csv_path, expr_by_sample_df)
top_50_genes_rfe   = globals().get("top_50_genes_rfe",   []) or load_gene_list(rfe_csv_path,   expr_by_sample_df)

print("LASSO50 present:", len(top_50_genes_lasso))
print("RFE50 present :", len(top_50_genes_rfe))

X_lasso50_full = expr_by_sample_df[top_50_genes_lasso].copy()
X_rfe50_full   = expr_by_sample_df[top_50_genes_rfe].copy()

# 2) Targets and fixed train/test split from metadata
y_str = meta_aligned_df[pam50_label_col].astype(str).str.strip()
le = LabelEncoder()
y_enc = pd.Series(le.fit_transform(y_str), index=y_str.index)
class_names = list(le.classes_)

train_mask = meta_aligned_df["Training.set"].astype(bool)
train_ids = y_enc.index[train_mask]
test_ids  = y_enc.index[~train_mask]

print("Shapes — X_lasso50_full:", X_lasso50_full.shape, " X_rfe50_full:", X_rfe50_full.shape)
print("Classes:", class_names)
print(f"Split: train={len(train_ids)}, test={len(test_ids)}")

## Evaluate

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import pandas as pd

def train_eval_feature_set(X_full, name):
    X_tr, X_te = X_full.loc[train_ids], X_full.loc[test_ids]
    y_tr, y_te = y_enc.loc[train_ids], y_enc.loc[test_ids]

    results = []
    models = model_zoo(len(class_names))
    for mname, model in models.items():
        model.fit(X_tr, y_tr)
        y_pred = model.predict(X_te)
        acc = accuracy_score(y_te, y_pred)
        f1m = f1_score(y_te, y_pred, average="macro")
        results.append((mname, acc, f1m))
        print(f"\n=== {name} · {mname} ===")
        print(classification_report(y_te, y_pred, target_names=class_names, digits=3))
        print(confusion_matrix(y_te, y_pred))
    res = pd.DataFrame(results, columns=["Model","Accuracy","MacroF1"]).set_index("Model").sort_values("MacroF1", ascending=False)
    print(f"\n{name} summary:\n", res)
    return res

# CLASSIFICATION MODELS

## 1. For LASSO50

In [ ]:
lasso_summary = train_eval_feature_set(X_lasso50_full, "LASSO50")

## 2. For RFE50

In [ ]:
rfe_summary = train_eval_feature_set(X_rfe50_full, "RFE50")

In [ ]:
# 1) combine into one tidy table
comparison_df = (
    pd.concat({'LASSO50': lasso_summary, 'RFE50': rfe_summary}, names=['FeatureSet'])
      .reset_index()  # columns: FeatureSet, Model, Accuracy, MacroF1
)
comparison_df = comparison_df.sort_values(['MacroF1','Accuracy'], ascending=False)
print("== Combined summary (sorted by MacroF1) ==")
display(comparison_df)

# 2) bar chart of Macro-F1
plt.figure(figsize=(8,4))
for fs in ['LASSO50','RFE50']:
    sub = comparison_df[comparison_df['FeatureSet']==fs]
    plt.bar(sub['Model'] + " ("+fs+")", sub['MacroF1'])
plt.xticks(rotation=45, ha='right')
plt.ylabel("Macro-F1")
plt.title("Subtype classification: Macro-F1 by model & feature set")
plt.tight_layout()
plt.show()

# 3) pick best model name per feature set
best_lasso_model = lasso_summary['MacroF1'].idxmax()
best_rfe_model   = rfe_summary['MacroF1'].idxmax()
print("Best LASSO50 model:", best_lasso_model)
print("Best RFE50 model  :", best_rfe_model)

## helper to refit + evaluate + plot confusion matrix

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score
import matplotlib.pyplot as plt

def refit_and_eval(X_full, model_name, label_note):
    # data split
    X_tr, X_te = X_full.loc[train_ids], X_full.loc[test_ids]
    y_tr, y_te = y_enc.loc[train_ids], y_enc.loc[test_ids]

    # get a fresh instance of the chosen model
    clf = model_zoo(len(class_names))[model_name]
    clf.fit(X_tr, y_tr)

    # predictions
    y_pred = clf.predict(X_te)

    # metrics
    acc = accuracy_score(y_te, y_pred)
    f1m = f1_score(y_te, y_pred, average="macro")
    print(f"\n== {label_note} · {model_name} ==")
    print(f"Accuracy: {acc:.3f} | Macro-F1: {f1m:.3f}\n")
    print(classification_report(y_te, y_pred, target_names=class_names, digits=3))

    # confusion matrix
    cm = confusion_matrix(y_te, y_pred, labels=range(len(class_names)))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(6, 5))
    disp.plot(ax=ax, xticks_rotation=45, colorbar=False)
    plt.title(f"{label_note} · {model_name}")
    plt.tight_layout()
    plt.show()

    return clf, y_pred

# Refit/evaluate the best LASSO50 model

In [ ]:
best_lasso_model  # should already be defined from the summary step
clf_lasso_best, ypred_lasso = refit_and_eval(X_lasso50_full, best_lasso_model, "LASSO50")

## Refit/evaluate the best RFE50 model

In [ ]:
best_rfe_model  # should already be defined from the summary step
clf_rfe_best, ypred_rfe = refit_and_eval(X_rfe50_full, best_rfe_model, "RFE50")

## Excluding “unclassified”

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# which string is used for that class in your labels?
uncl_token = "unclassified"

# mask test set to drop "unclassified"
test_labels_series = meta_aligned_df.loc[test_ids, pam50_label_col].astype(str).str.strip()
keep_mask = test_labels_series.str.lower() != uncl_token

X_te_clean = X_rfe50_full.loc[test_ids[keep_mask.values]]
y_te_clean = y_enc.loc[test_ids[keep_mask.values]]

# re-predict with your already-fitted champion (clf_rfe_best)
y_pred_clean = clf_rfe_best.predict(X_te_clean)

# restrict names/labels to the kept classes
kept_class_names = [c for c in class_names if c.lower() != uncl_token]
kept_label_ids   = [int(v) for v in le.transform(kept_class_names)]

print(classification_report(y_te_clean, y_pred_clean, target_names=class_names, labels=kept_label_ids, digits=3))

cm = confusion_matrix(y_te_clean, y_pred_clean, labels=kept_label_ids)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=kept_class_names)
fig, ax = plt.subplots(figsize=(6,5))
disp.plot(ax=ax, xticks_rotation=45, colorbar=False)
plt.title("RFE50 · XGB (unclassified removed)")
plt.tight_layout()
plt.show()

# EXPLAINABLE ML

In [ ]:
# interpret RFE50 first
X_feat = X_rfe50_full
X_tr, X_te = X_feat.loc[train_ids], X_feat.loc[test_ids]
y_tr, y_te = y_enc.loc[train_ids], y_enc.loc[test_ids]

models_rfe = model_zoo(num_classes=len(class_names))  # from earlier
for name in models_rfe:
    models_rfe[name].fit(X_tr, y_tr)

list(models_rfe.keys())

## 5 Models are fitted on RFE50

In [ ]:
X_tr_rfe = X_rfe50_full.loc[train_ids]
X_te_rfe = X_rfe50_full.loc[test_ids]
y_tr     = y_enc.loc[train_ids]

models_rfe = model_zoo(num_classes=len(class_names))
for name, m in models_rfe.items():
    m.fit(X_tr_rfe, y_tr)

# pick which class to visualize
class_to_plot = "LumA"  # change to "LumB", "Basal", "HER2", "Normal"
class_idx = class_names.index(class_to_plot)

## SHAP for tree models + RFE (Random Forest, XGBoost)

In [ ]:
# ---- RF (RFE50) ----
explainer_rf_rfe = shap.TreeExplainer(models_rfe["RF"])
shap_vals_rf_rfe = explainer_rf_rfe.shap_values(X_te_rfe)
if isinstance(shap_vals_rf_rfe, list):
    shap_vals_rf_rfe = shap_vals_rf_rfe[class_idx]

shap.summary_plot(shap_vals_rf_rfe, X_te_rfe, plot_type="bar", show=True)
shap.summary_plot(shap_vals_rf_rfe, X_te_rfe, show=True)

In [ ]:
# ---- XGB (RFE50) ----
explainer_xgb_rfe = shap.TreeExplainer(models_rfe["XGB"])
shap_vals_xgb_rfe = explainer_xgb_rfe.shap_values(X_te_rfe)
if isinstance(shap_vals_xgb_rfe, list):
    shap_vals_xgb_rfe = shap_vals_xgb_rfe[class_idx]

shap.summary_plot(shap_vals_xgb_rfe, X_te_rfe, plot_type="bar", show=True)
shap.summary_plot(shap_vals_xgb_rfe, X_te_rfe, show=True)

rng = np.random.default_rng(42)
bg_idx   = rng.choice(len(X_tr_rfe), size=min(200, len(X_tr_rfe)), replace=False)
eval_idx = rng.choice(len(X_te_rfe), size=min(600, len(X_te_rfe)), replace=False)
X_bg     = X_tr_rfe.iloc[bg_idx]
X_eval   = X_te_rfe.iloc[eval_idx]

# ---- Logistic Regression (RFE50) ----
expl_lr = shap.KernelExplainer(models_rfe["LogReg"].predict_proba, X_bg)
shap_vals_lr = expl_lr.shap_values(X_eval)            # list per class
shap.summary_plot(shap_vals_lr[class_idx], X_eval, plot_type="bar", show=True)
shap.summary_plot(shap_vals_lr[class_idx], X_eval, show=True)

# ---- SVM (RFE50) ----
expl_svm = shap.KernelExplainer(models_rfe["SVM"].predict_proba, X_bg)
shap_vals_svm = expl_svm.shap_values(X_eval)
shap.summary_plot(shap_vals_svm[class_idx], X_eval, plot_type="bar", show=True)
shap.summary_plot(shap_vals_svm[class_idx], X_eval, show=True)

# ---- MLP (RFE50) ----
expl_mlp = shap.KernelExplainer(models_rfe["MLP"].predict_proba, X_bg)
shap_vals_mlp = expl_mlp.shap_values(X_eval)
shap.summary_plot(shap_vals_mlp[class_idx], X_eval, plot_type="bar", show=True)
shap.summary_plot(shap_vals_mlp[class_idx], X_eval, show=True)

## SHAP for tree models + LASSO (Random Forest, XGBoost)

In [ ]:
# Data splits
X_tr_lasso = X_lasso50_full.loc[train_ids]
X_te_lasso = X_lasso50_full.loc[test_ids]
y_tr       = y_enc.loc[train_ids]

# Fit only the two tree models (reuse your model_zoo for identical settings)
rf_lasso  = model_zoo(len(class_names))["RF"]
xgb_lasso = model_zoo(len(class_names))["XGB"]

rf_lasso.fit(X_tr_lasso, y_tr)
xgb_lasso.fit(X_tr_lasso, y_tr)

# Choose which subtype to plot (change as you like)
class_to_plot = "LumA"       # e.g., "LumB", "Basal", "HER2", "Normal"
class_idx = class_names.index(class_to_plot)
print("Plotting class:", class_to_plot, "-> index", class_idx)

In [ ]:
# --- Random Forest (LASSO50) ---
explainer_rf_lasso = shap.TreeExplainer(rf_lasso)
sv_rf_lasso = explainer_rf_lasso.shap_values(X_te_lasso)
if isinstance(sv_rf_lasso, list):   # multiclass -> list per class
    sv_rf_lasso = sv_rf_lasso[class_idx]

shap.summary_plot(sv_rf_lasso, X_te_lasso, plot_type="bar", show=True)
shap.summary_plot(sv_rf_lasso, X_te_lasso, show=True)

In [ ]:
# --- XGBoost (LASSO50) ---
explainer_xgb_lasso = shap.TreeExplainer(xgb_lasso)
sv_xgb_lasso = explainer_xgb_lasso.shap_values(X_te_lasso)
if isinstance(sv_xgb_lasso, list):
    sv_xgb_lasso = sv_xgb_lasso[class_idx]

shap.summary_plot(sv_xgb_lasso, X_te_lasso, plot_type="bar", show=True)
shap.summary_plot(sv_xgb_lasso, X_te_lasso, show=True)

## 2. PDP

In [ ]:
# Data splits
X_tr_rfe, X_te_rfe = X_rfe50_full.loc[train_ids],  X_rfe50_full.loc[test_ids]
X_tr_las, X_te_las = X_lasso50_full.loc[train_ids], X_lasso50_full.loc[test_ids]
y_tr, y_te = y_enc.loc[train_ids], y_enc.loc[test_ids]

# Fresh model sets
models_rfe   = model_zoo(num_classes=len(class_names))
models_lasso = model_zoo(num_classes=len(class_names))

for m in models_rfe.values():   m.fit(X_tr_rfe, y_tr)
for m in models_lasso.values(): m.fit(X_tr_las, y_tr)

list(models_rfe.keys())  # ['LogReg','SVM','RF','XGB','MLP']

## LUMA

In [ ]:
# choose the subtype to visualize on the y-axis (probability of this class)
class_to_plot = "LumA"                       # e.g., "LumB", "Basal", "HER2", "Normal"
cidx = class_names.index(class_to_plot)      # class_names came from your LabelEncoder

In [ ]:
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
import numpy as np
import matplotlib.pyplot as plt

def _unwrap(est):
    return est.named_steps[list(est.named_steps)[-1]] if hasattr(est, "named_steps") else est

def _resp(est):
    est = _unwrap(est)
    return "predict_proba" if hasattr(est, "predict_proba") else "decision_function"

def topk_features(estimator, X_ref, y_ref=None, k=9):
    fin = _unwrap(estimator)
    if hasattr(fin, "feature_importances_"):
        w = np.asarray(fin.feature_importances_)
    elif hasattr(fin, "coef_"):
        w = np.mean(np.abs(np.asarray(fin.coef_)), axis=0)  # multiclass → avg abs
    else:
        pi = permutation_importance(estimator, X_ref, y_ref, n_repeats=5, random_state=42, scoring="accuracy")
        w = pi.importances_mean
    order = np.argsort(w)[::-1][:k]
    return X_ref.columns[order].tolist()

def plot_pdp_block_multi(model, X_ref, title, k=9, n_cols=3, grid=50):
    feats = topk_features(model, X_ref, y_ref=y_tr, k=k)
    disp = PartialDependenceDisplay.from_estimator(
        model, X_ref, features=feats, kind="both",
        target=cidx, response_method=_resp(model),
        grid_resolution=grid, n_jobs=-1
    )
    # match the old look: tight grid, minimal whitespace
    n_rows = int(np.ceil(len(feats)/n_cols))
    fig = plt.gcf()
    fig.set_size_inches(5*n_cols, 2.8*n_rows)
    fig.suptitle(f"{title} — PDP + ICE (top {k} features)", y=0.98, fontsize=12)
    plt.tight_layout(rect=[0,0,1,0.95]); plt.show()
    return feats

In [ ]:
# assume models_rfe are already fitted on X_tr_rfe, y_tr
rfe_lr_feats  = plot_pdp_block_multi(models_rfe["LogReg"], X_tr_rfe, f"Logistic Regression (RFE) — {class_to_plot}")
rfe_rf_feats  = plot_pdp_block_multi(models_rfe["RF"],     X_tr_rfe, f"Random Forest (RFE) — {class_to_plot}")
rfe_svm_feats = plot_pdp_block_multi(models_rfe["SVM"],    X_tr_rfe, f"SVM (RFE) — {class_to_plot}")
rfe_xgb_feats = plot_pdp_block_multi(models_rfe["XGB"],    X_tr_rfe, f"XGBoost (RFE) — {class_to_plot}")
rfe_mlp_feats = plot_pdp_block_multi(models_rfe["MLP"],    X_tr_rfe, f"MLP (RFE) — {class_to_plot}")

In [ ]:
# assume models_lasso are already fitted on X_tr_las, y_tr
las_lr_feats  = plot_pdp_block_multi(models_lasso["LogReg"], X_tr_las, f"Logistic Regression (LASSO) — {class_to_plot}")
las_rf_feats  = plot_pdp_block_multi(models_lasso["RF"],     X_tr_las, f"Random Forest (LASSO) — {class_to_plot}")
las_svm_feats = plot_pdp_block_multi(models_lasso["SVM"],    X_tr_las, f"SVM (LASSO) — {class_to_plot}")
las_xgb_feats = plot_pdp_block_multi(models_lasso["XGB"],    X_tr_las, f"XGBoost (LASSO) — {class_to_plot}")
las_mlp_feats = plot_pdp_block_multi(models_lasso["MLP"],    X_tr_las, f"MLP (LASSO) — {class_to_plot}")

## BASAL

In [ ]:
# choose the subtype to visualize on the y-axis (probability of this class)
class_to_plot = "Basal"                       # e.g., "LumB", "Basal", "HER2", "Normal"
cidx = class_names.index(class_to_plot)      # class_names came from your LabelEncoder

In [ ]:
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
import numpy as np
import matplotlib.pyplot as plt

def _unwrap(est):
    return est.named_steps[list(est.named_steps)[-1]] if hasattr(est, "named_steps") else est

def _resp(est):
    est = _unwrap(est)
    return "predict_proba" if hasattr(est, "predict_proba") else "decision_function"

def topk_features(estimator, X_ref, y_ref=None, k=9):
    fin = _unwrap(estimator)
    if hasattr(fin, "feature_importances_"):
        w = np.asarray(fin.feature_importances_)
    elif hasattr(fin, "coef_"):
        w = np.mean(np.abs(np.asarray(fin.coef_)), axis=0)  # multiclass → avg abs
    else:
        pi = permutation_importance(estimator, X_ref, y_ref, n_repeats=5, random_state=42, scoring="accuracy")
        w = pi.importances_mean
    order = np.argsort(w)[::-1][:k]
    return X_ref.columns[order].tolist()

def plot_pdp_block_multi(model, X_ref, title, k=9, n_cols=3, grid=50):
    feats = topk_features(model, X_ref, y_ref=y_tr, k=k)
    disp = PartialDependenceDisplay.from_estimator(
        model, X_ref, features=feats, kind="both",
        target=cidx, response_method=_resp(model),
        grid_resolution=grid, n_jobs=-1
    )
    # match the old look: tight grid, minimal whitespace
    n_rows = int(np.ceil(len(feats)/n_cols))
    fig = plt.gcf()
    fig.set_size_inches(5*n_cols, 2.8*n_rows)
    fig.suptitle(f"{title} — PDP + ICE (top {k} features)", y=0.98, fontsize=12)
    plt.tight_layout(rect=[0,0,1,0.95]); plt.show()
    return feats

In [ ]:
# assume models_rfe are already fitted on X_tr_rfe, y_tr
rfe_lr_feats  = plot_pdp_block_multi(models_rfe["LogReg"], X_tr_rfe, f"Logistic Regression (RFE) — {class_to_plot}")
rfe_rf_feats  = plot_pdp_block_multi(models_rfe["RF"],     X_tr_rfe, f"Random Forest (RFE) — {class_to_plot}")
rfe_svm_feats = plot_pdp_block_multi(models_rfe["SVM"],    X_tr_rfe, f"SVM (RFE) — {class_to_plot}")
rfe_xgb_feats = plot_pdp_block_multi(models_rfe["XGB"],    X_tr_rfe, f"XGBoost (RFE) — {class_to_plot}")
rfe_mlp_feats = plot_pdp_block_multi(models_rfe["MLP"],    X_tr_rfe, f"MLP (RFE) — {class_to_plot}")

In [ ]:
# assume models_lasso are already fitted on X_tr_las, y_tr
las_lr_feats  = plot_pdp_block_multi(models_lasso["LogReg"], X_tr_las, f"Logistic Regression (LASSO) — {class_to_plot}")
las_rf_feats  = plot_pdp_block_multi(models_lasso["RF"],     X_tr_las, f"Random Forest (LASSO) — {class_to_plot}")
las_svm_feats = plot_pdp_block_multi(models_lasso["SVM"],    X_tr_las, f"SVM (LASSO) — {class_to_plot}")
las_xgb_feats = plot_pdp_block_multi(models_lasso["XGB"],    X_tr_las, f"XGBoost (LASSO) — {class_to_plot}")
las_mlp_feats = plot_pdp_block_multi(models_lasso["MLP"],    X_tr_las, f"MLP (LASSO) — {class_to_plot}")

## LumB

In [ ]:
# choose the subtype to visualize on the y-axis (probability of this class)
class_to_plot = "LumB"                       # e.g., "LumB", "Basal", "HER2", "Normal"
cidx = class_names.index(class_to_plot)      # class_names came from your LabelEncoder

In [ ]:
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
import numpy as np
import matplotlib.pyplot as plt

def _unwrap(est):
    return est.named_steps[list(est.named_steps)[-1]] if hasattr(est, "named_steps") else est

def _resp(est):
    est = _unwrap(est)
    return "predict_proba" if hasattr(est, "predict_proba") else "decision_function"

def topk_features(estimator, X_ref, y_ref=None, k=9):
    fin = _unwrap(estimator)
    if hasattr(fin, "feature_importances_"):
        w = np.asarray(fin.feature_importances_)
    elif hasattr(fin, "coef_"):
        w = np.mean(np.abs(np.asarray(fin.coef_)), axis=0)  # multiclass → avg abs
    else:
        pi = permutation_importance(estimator, X_ref, y_ref, n_repeats=5, random_state=42, scoring="accuracy")
        w = pi.importances_mean
    order = np.argsort(w)[::-1][:k]
    return X_ref.columns[order].tolist()

def plot_pdp_block_multi(model, X_ref, title, k=9, n_cols=3, grid=50):
    feats = topk_features(model, X_ref, y_ref=y_tr, k=k)
    disp = PartialDependenceDisplay.from_estimator(
        model, X_ref, features=feats, kind="both",
        target=cidx, response_method=_resp(model),
        grid_resolution=grid, n_jobs=-1
    )
    # match the old look: tight grid, minimal whitespace
    n_rows = int(np.ceil(len(feats)/n_cols))
    fig = plt.gcf()
    fig.set_size_inches(5*n_cols, 2.8*n_rows)
    fig.suptitle(f"{title} — PDP + ICE (top {k} features)", y=0.98, fontsize=12)
    plt.tight_layout(rect=[0,0,1,0.95]); plt.show()
    return feats

In [ ]:
# assume models_rfe are already fitted on X_tr_rfe, y_tr
rfe_lr_feats  = plot_pdp_block_multi(models_rfe["LogReg"], X_tr_rfe, f"Logistic Regression (RFE) — {class_to_plot}")
rfe_rf_feats  = plot_pdp_block_multi(models_rfe["RF"],     X_tr_rfe, f"Random Forest (RFE) — {class_to_plot}")
rfe_svm_feats = plot_pdp_block_multi(models_rfe["SVM"],    X_tr_rfe, f"SVM (RFE) — {class_to_plot}")
rfe_xgb_feats = plot_pdp_block_multi(models_rfe["XGB"],    X_tr_rfe, f"XGBoost (RFE) — {class_to_plot}")
rfe_mlp_feats = plot_pdp_block_multi(models_rfe["MLP"],    X_tr_rfe, f"MLP (RFE) — {class_to_plot}")

In [ ]:
# assume models_lasso are already fitted on X_tr_las, y_tr
las_lr_feats  = plot_pdp_block_multi(models_lasso["LogReg"], X_tr_las, f"Logistic Regression (LASSO) — {class_to_plot}")
las_rf_feats  = plot_pdp_block_multi(models_lasso["RF"],     X_tr_las, f"Random Forest (LASSO) — {class_to_plot}")
las_svm_feats = plot_pdp_block_multi(models_lasso["SVM"],    X_tr_las, f"SVM (LASSO) — {class_to_plot}")
las_xgb_feats = plot_pdp_block_multi(models_lasso["XGB"],    X_tr_las, f"XGBoost (LASSO) — {class_to_plot}")
las_mlp_feats = plot_pdp_block_multi(models_lasso["MLP"],    X_tr_las, f"MLP (LASSO) — {class_to_plot}")